# Import Libraries

In [1]:
from analysis_functions import *

print('Loaded')

2025-03-11 01:49:07.410522: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-11 01:49:07.415913: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-11 01:49:07.430217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-11 01:49:07.454397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-11 01:49:07.461230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 01:49:08.695650: W tensorflow/compiler/tf2tensorrt/utils/p

Loaded


# IQ Processor

In [5]:

features_dir = "../../features/" # Define the relative directory path for the features folder
os.makedirs(features_dir, exist_ok=True) # Ensure the features directory exists, create it if it doesn't

# ---------- IMPORT SIGNAL ---------- 

data_dir = "../../data/" # Define the relative directory path for the data folder
suffixes = [f"{i:03d}" for i in range(170)]

sufix = 1           # 001 - 170
signal = 90         #0 - 9999 (Signals)
fs = 25e6           #sample frequency
number_peaks=10     #number of peaks

samples = load_samples(data_dir,f"{sufix:03d}")

# Obtain I[0] and Q[1] signals
i_signal = samples[signal,:,0]
q_signal = samples[signal,:,1]



print(f"Sample_{sufix:03d}:{signal:04d} imported")

# ---------- end import signal ---------- 

normalize = False

# ---------- COMPUTE IQ SIGNALS FEATURES  ---------- 
features_mag, features_phase = iq_features_processor(i_signal, q_signal, fs, number_peaks, normalize =normalize)

# ---------- end compute iq signals features ---------- 

# ---------- COMPUTE DATASET AND METADATA FOR EACH TRANSMITTER  ---------- 

features, metadata = create_feature_matrix(features_mag, features_phase, sat_label=56, transmitter_label=7, feature_counter=1, normalize = normalize)

# Print results for verification
print("Feature Matrix Shape:", features.shape)
print("Feature Matrix Name:", metadata["Feature_Matrix_Name"])
print("Features Names:", metadata["Colum_Feature_Names"])
print("Metadata:", json.dumps(metadata, indent=2))

# ---------- end compute dataset and metadata for each transmitter  ---------- 

# Save the feature matrix and metadata to the features folder
np.save(os.path.join(features_dir, f"{metadata['Feature_Matrix_Name']}.npy"), features)
with open(os.path.join(features_dir, f"{metadata['Feature_Matrix_Name']}_metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)
    
print(features)


Sample_001:0090 imported
Feature Matrix Shape: (20, 9)
Feature Matrix Name: features_1
Features Names: ['power', 'entropy', 'spectral_centroid', 'psd_mean', 'psd_variance', 'l_band', 'h_band', 'frequency_peak', 'Magnitude_peak']
Metadata: {
  "Satellite_Label": 56,
  "Transmitter_Label": 7,
  "Feature_Matrix_Name": "features_1",
  "Normalized": "False",
  "Colum_Feature_Names": [
    "power",
    "entropy",
    "spectral_centroid",
    "psd_mean",
    "psd_variance",
    "l_band",
    "h_band",
    "frequency_peak",
    "Magnitude_peak"
  ],
  "Rows": [
    "Magnitude_Band_1",
    "Magnitude_Band_2",
    "Magnitude_Band_3",
    "Magnitude_Band_4",
    "Magnitude_Band_5",
    "Magnitude_Band_6",
    "Magnitude_Band_7",
    "Magnitude_Band_8",
    "Magnitude_Band_9",
    "Magnitude_Band_10",
    "Phase_Band_1",
    "Phase_Band_2",
    "Phase_Band_3",
    "Phase_Band_4",
    "Phase_Band_5",
    "Phase_Band_6",
    "Phase_Band_7",
    "Phase_Band_8",
    "Phase_Band_9",
    "Phase_Band_10"

## PRUEBA DE GUARDADO CON TENSOR FLOW

In [28]:
import tensorflow as tf
import json
import os

def save_features_with_tf(features, metadata, features_dir="../../features/"):
    """
    Saves the features matrix and metadata using TensorFlow's TFRecord format.
    
    Args:
        features (np.ndarray): Concatenated feature matrix (magnitude + phase).
        metadata (dict): Dictionary containing metadata (labels, feature names, etc.).
        features_dir (str, optional): Directory to save the files. Defaults to "../../features/".
    """
    # Ensure the features directory exists
    os.makedirs(features_dir, exist_ok=True)
    
    # Define the output file path
    file_path = os.path.join(features_dir, f"{metadata['Feature_Matrix_Name']}.tfrecord")
    
    # Convert NumPy array to TensorFlow tensor
    features_tensor = tf.convert_to_tensor(features, dtype=tf.float32)
    
    # Serialize metadata to a JSON string
    metadata_json = json.dumps(metadata)
    
    # Define TFRecord feature helper functions
    def _float_feature(value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=value))
    
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
    
    # Flatten the features tensor for serialization
    flat_features = features_tensor.numpy().flatten()
    
    # Create a TFRecord example
    example = tf.train.Example(features=tf.train.Features(feature={
        'features': _float_feature(flat_features),
        'feature_shape': _int64_feature([features.shape[0], features.shape[1]]),  # Use int64 for shape
        'metadata': _bytes_feature(metadata_json)
    }))
    
    # Write the example to a TFRecord file
    with tf.io.TFRecordWriter(file_path) as writer:
        writer.write(example.SerializeToString())
    print(f"Saved to {file_path}")

def parse_tfrecord(example_proto):
    """
    Parses a single TFRecord example back into features and metadata.
    """
    feature_description = {
        'features': tf.io.FixedLenFeature([180], tf.float32),  # Adjust size based on your data (e.g., 20 * 9)
        'feature_shape': tf.io.FixedLenFeature([2], tf.int64),  # Parse shape as int64
        'metadata': tf.io.FixedLenFeature([], tf.string)
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    
    # Reshape features back to original dimensions using int64 shape
    features = tf.reshape(parsed['features'], parsed['feature_shape'])
    metadata_json = parsed['metadata']
    return features, metadata_json


# Save the features and metadata
save_features_with_tf(features, metadata)

# Load and verify
dataset = tf.data.TFRecordDataset("../../features/features_001.tfrecord")
parsed_dataset = dataset.map(parse_tfrecord)

for features_tensor, metadata_json in parsed_dataset.take(1):
    print("Loaded Features Shape:", features_tensor.shape)
    print("Loaded Metadata:", json.loads(metadata_json.numpy().decode()))

Saved to ../../features/features_1.tfrecord
Loaded Features Shape: (20, 9)
Loaded Metadata: {'Feature_Matrix_Name': 'features_001', 'Satellite_Label': 1, 'Transmitter_Label': 1, 'Feature_Names': ['power', 'entropy', 'spectral_centroid', 'psd_mean', 'psd_variance', 'l_band', 'h_band', 'frequency_peak', 'Magnitude_peak'], 'Rows': ['Magnitude_Band_1', 'Magnitude_Band_2', 'Magnitude_Band_3', 'Magnitude_Band_4', 'Magnitude_Band_5', 'Magnitude_Band_6', 'Magnitude_Band_7', 'Magnitude_Band_8', 'Magnitude_Band_9', 'Magnitude_Band_10', 'Phase_Band_1', 'Phase_Band_2', 'Phase_Band_3', 'Phase_Band_4', 'Phase_Band_5', 'Phase_Band_6', 'Phase_Band_7', 'Phase_Band_8', 'Phase_Band_9', 'Phase_Band_10']}


In [4]:
def bool_to_string(value):
    """
    Convierte un valor booleano a su representación en cadena ("True" o "False").
    
    Args:
        value (bool): El valor booleano a convertir.
    
    Returns:
        str: La representación en cadena del valor booleano.
    """
    return str(value)

# Ejemplos de uso
bool_value_1 = True
bool_value_2 = False

print(bool_to_string(bool_value_1))  # Salida: "True"
print(bool_to_string(bool_value_2))  # Salida: "False"

True
False
